In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("en openfoodfacts org products.csv")

In [3]:
df.drop(df[df['pnns_groups_1'] == "unknown"].index, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4735 entries, 6971 to 13218
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   code                                     4735 non-null   int64  
 1   product_name                             4475 non-null   object 
 2   quantity                                 3582 non-null   object 
 3   packaging_en                             1798 non-null   object 
 4   brands_en                                4488 non-null   object 
 5   categories_en                            4653 non-null   object 
 6   origins_en                               888 non-null    object 
 7   labels_en                                2494 non-null   object 
 8   countries_en                             4735 non-null   object 
 9   nutriscore_grade                         4650 non-null   object 
 10  pnns_groups_1                            4650 non

In [5]:
df.dropna(subset=['pnns_groups_2'], inplace=True)

In [6]:
df.to_csv("cleaned_data.csv")

In [7]:
df_co2 = pd.read_csv('https://raw.githubusercontent.com/Phi-project3/finalproject/refs/heads/main/dossier/donnees/source/category_schema/products_category_schema_v1.csv', sep=';')

df_co2.dropna(subset=['category_name'], inplace=True)

df_merged = df.merge(df_co2[['category_name', 'average']], how='left', left_on='pnns_groups_2', right_on='category_name')

df_merged.drop(columns=['category_name', 'carbon-footprint_100g', 'carbon-footprint-from-meat-or-fish_100g'], inplace=True)

df_merged.rename(columns={'average': 'carbon-footprint_1kg'}, inplace=True)

df_merged.sample(5)

,code,product_name,quantity,packaging_en,brands_en,categories_en,origins_en,labels_en,countries_en,nutriscore_grade,pnns_groups_1,pnns_groups_2,food_groups_en,environmental_score_score,environmental_score_grade,main_category_en,carbon-footprint_1kg
3022,20221904,Hähnchen Nuggets Käse,450 g ℮bane,"Cardboard,Non-corrugated cardboard,LDPE 4 - Lo...","Gutknecht,Culinea","Meats and their products,Meat preparations,Mea...",NaN,"German Agricultural Society,de:dlg-jahrlich-pr...","Finland,Germany",d,Fish Meat Eggs,Meat,"Fish‚ Meat‚ Eggs,Meat,Poultry",22.0,e,Chicken nuggets,43.33
3072,5411188102809,"Alpro Fermentiertes Sojaprodukt, Heidelbeere, ...",0.5 kg,"Metal,Plastic,Recyclable Metals,Aluminium,Card...",Alpro,"Beverages and beverages preparations,Plant-bas...",NaN,"No gluten,Vegetarian,Vegan,European Vegetarian...","Belgium,Finland,France,Germany,Netherlands,Spa...",c,Beverages,Plant-based milk substitutes,"Beverages,Plant-based milk substitutes",49.0,c,Fermentiertes-sojaprodukt,0.90
391,6412500074063,Lakumix Original,NaN,NaN,panda,"Snacks,Sweet snacks,Confectioneries,Candies",NaN,NaN,Finland,e,Sugary snacks,Sweets,"Sugary snacks,Sweets",42.0,d,Candies,2.95
3186,3850102314126,Kras Tea Biscuits With Chocolate,300 g,"Plastic,Pp-polypropylene,Cardboard,Non-corruga...",krašdomaćica,"Snacks,Sweet snacks,Biscuits and cakes,Biscuit...",NaN,Halal,"Croatia,Finland,France,Italy",e,Sugary snacks,Sweets,"Sugary snacks,Sweets",33.0,d,fr:keksi-i-kolači,2.95
2299,6420256015308,Tupla double layer mint,48 g,NaN,Cloetta,"Snacks,Sweet snacks,Cocoa and its products,Con...",NaN,Rainforest Alliance,Finland,e,Sugary snacks,Biscuits and cakes,"Sugary snacks,Biscuits and cakes",NaN,NaN,Bars-covered-with-chocolate,2.50


In [8]:
df_merged.to_csv("merged_data.csv")

In [21]:
df_main = pd.read_csv("merged_data.csv")
df_update = pd.read_csv("finetuned_data.csv", sep=";")
 
df_main["code"] = df_main["code"].astype(str)
df_update["code"] = df_update["code"].astype(str)
 
df_main_updated = df_main.merge(df_update, on='code', how='left')
df_main_updated
 
# Update only where new categories exist
df_main_updated['pnns_groups_2'] = df_main_updated['new_category (pnns_groups_2)'].combine_first(df_main_updated['pnns_groups_2'])
 
# Drop the helper column
df_main_updated.drop(columns=['new_category (pnns_groups_2)'], inplace=True)
 
df_main_updated.to_csv("updated_products.csv", index=False)
 
df_main_updated.sample(3)

,Unnamed: 0,code,product_name,quantity,packaging_en,brands_en,categories_en,origins_en,labels_en,countries_en,nutriscore_grade,pnns_groups_1,pnns_groups_2,food_groups_en,environmental_score_score,environmental_score_grade,main_category_en,carbon-footprint_1kg
345,345,5690845002786,Kookos-sitruuna Skyr,400 g,NaN,Isey,"Dairies,Fermented foods,Fermented milk product...",NaN,NaN,Finland,b,Milk and dairy products,Dairy desserts,"Milk and dairy products,Dairy desserts",78.0,a,Fruit skyrs,2.05
4508,4506,6415712601379,Juustopasta-ateria,160g,fi:komposiitti,Rainbow,"Meals,Pasta dishes",NaN,NaN,Finland,unknown,Composite foods,One-dish meals,"Composite foods,One-dish meals",NaN,NaN,Pasta dishes,2.95
4401,4399,20604493,risoniateria,0.25 kg,"Plastic,Box",Delicorner-eataway,"Meals,Pasta dishes",NaN,NaN,Finland,d,Composite foods,One-dish meals,"Composite foods,One-dish meals",NaN,NaN,Pasta dishes,2.95
